### FT comments scraper 2.0

In [2]:
import requests
import time
import json
import pandas as pd
import pickle
import sqlite3

Second trial with FT comments section scraping
- retrieve the article ID of the ft via http://api.ft.com/content/search/v1 (with headers X-Api-Key:k_E_Y)
- retrieve corresponding livefyre comments ID via https://session-user-data.webservices.ft.com/v1/livefyre/get_lf_bootstrap?uuid=ARTICLE_ID
- retrieve comments via https://bootstrap.ft.fyre.co/bs3/ft.fyre.co/378157/LIVEFYRE_ID/init

In [ ]:
TOPICS = ['Syria','Gazprom','Brexit',
          'MH17','NATO','Trump','Assad','Georgia','IOC',
          'Doping','Putin','Merkel','Refugees','Afd','Le+Pen',
          'clinton','guns','iran','macron']

In [ ]:
def get_FT_ID(topic,offset=0):
    FT_ID =[]
    url = 'http://api.ft.com/content/search/v1'
    headers = {'X-Api-Key': 'qeuya9utyhcu7p6fxv429t4f'}
    body={"queryString": topic,"queryContext" : {"curations" : ["ARTICLES"]}, "resultContext": {"offset": offset}}
    try:
        response = requests.post(url,data=json.dumps(body),headers=headers, timeout=5)
        data = json.loads(response.content)
        FT_ID = [(topic,resp['id']) for resp in data['results'][0]['results']]
    except Exception as e:
        print e,topic
        
    return FT_ID

In [ ]:
def retrieve_FT_IDS_per_topic(topic="russia"):
    L = []
    i=0
    
    while i<4000:
        L+=get_FT_ID(topic,i)
        print topic,len(L)
        i+=100
    return L

In [ ]:
def retrieve_all_FT_IDS():
    L=[]
    for t in TOPICS:
        L+=retrieve_FT_IDS_per_topic(t)
    return L
        

In [ ]:
def get_all_FT_IDS():
    L_all = retrieve_all_FT_IDS()
    df = pd.DataFrame(L_all)
    df.columns=['topic','FT_ID']
    df = df.groupby(['FT_ID'])['topic'].apply(list)
    df_grouped = pd.DataFrame(df)
    df_grouped = df_grouped.reset_index()
    df_grouped.to_csv('ALL_FT_IDS.csv')

In [ ]:
df_grouped.head()

In [4]:
def retrieve_Livefyre_comment(ft_ID):
    comment_data =""
    url_FT_to_Livefyre = 'https://session-user-data.webservices.ft.com/v1/livefyre/get_lf_bootstrap?uuid='
    try:
        response_livefyreID = (requests.get(url_FT_to_Livefyre+str(ft_ID),timeout=5)).content
        liverfyre_comments_url =   (json.loads(response_livefyreID))['url'].replace('bootstrap.html','init').replace('https','http')
        response_livefyre_comment = requests.get(liverfyre_comments_url,timeout=5)
        comment_data = json.loads(response_livefyre_comment.content)
        
    except Exception as e:
        print "error"
    
    return (response_livefyreID,comment_data['collectionSettings']['title'],comment_data[u'headDocument'])

In [8]:
def get_ALL_COMMENTS():
    df = pd.read_csv('ALL_FT_IDS.csv',index_col=0)
    ALL_FT_IDS = df.FT_ID.tolist()
    ALL_FT_IDS.sort()
    LIVEFYRE = []
    for i,ft_id in enumerate(ALL_FT_IDS):
        if i>=20500:

            try:
                comment_data = retrieve_Livefyre_comment(ft_id)
                LIVEFYRE.append((ft_id,comment_data[0],comment_data[1],comment_data[2]))
            except Exception as e:
                print e


            if i%500==0:

                df_checkpoint = pd.DataFrame(LIVEFYRE,columns=['FT_ID','LIVEFYRE_ID','title','comments'])
                df_checkpoint.to_csv('df_'+str(i)+'.csv', encoding='utf-8')
                print "Checkpoint at : ", i*100.0/len(ALL_FT_IDS)

    

In [9]:
get_ALL_COMMENTS()

Checkpoint at :  47.4690872042
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be inte

error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be int

'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must b

'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collection

'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices m

...

error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
Checkpoint at :  91.4648265642
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
e

error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not

error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'

'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'colle

'collectionSettings'
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
error
string indices must be integers, not str
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
'collectionSettings'
'collectionSettings'
error
string indices must be integers, not str
